In [3]:

import pandas as pd
pd.set_option('display.max_columns', None)
from pathlib import Path
import json
dataset_path = Path("/data/dzeiberg/mave_calibration/data/Findlay_VHL_SGE/")
assert dataset_path.exists()
data = pd.read_excel(dataset_path/ "raw" / "41588_2024_1800_MOESM4_ESM.xlsx",header=2)

AUTHOR_TRANSCRIPT = "ENTS000000256474"
data = data.assign(author_transcript = AUTHOR_TRANSCRIPT)
data = data.rename(columns = {
    'hg38_pos' : "POSITION",
    'ref' : "REF",
    'alt' : "ALT",
    'chrom' : "CHROM"})

In [4]:
data

POSITION REF ALT     cHGVS   pHGVS  function_score_final   q_value  \
0     10141958   G   C  c.111G>C  p.E37D              0.227104  0.999801   
1     10141958   G   T  c.111G>T  p.E37D              0.480106  0.999801   
2     10141959   T   A  c.112T>A  p.S38T             -0.065021  0.736202   
3     10141959   T   G  c.112T>G  p.S38A              0.235124  0.999801   
4     10141960   C   A  c.113C>A  p.S38Y             -0.324690  0.124480   
...        ...  ..  ..       ...     ...                   ...       ...   
2263  10141956   G   T  c.109G>T  p.E37*              1.197395  0.986207   
2264  10141947   G   C  c.100G>C  p.G34R              1.199658  0.986207   
2265  10141951   C   A  c.104C>A  p.A35D              1.452510  0.986207   
2266  10141936   G   T   c.89G>T  p.G30V              1.506749  0.986207   
2267  10141951   C   T  c.104C>T  p.A35V              1.525030  0.986207   

      rna_score  rna_score_day_20  delta_rna function_class sge_region Intron  \
0      0.146014          0.049020  -0.096993        Neutral    exon 1b    NaN   
1      0.496812          0.314450  -0.182362        Neutral    exon 1b    NaN   
2     -0.072035         -0.281796  -0.209761        Neutral    exon 1b    NaN   
3     -2.064347         -1.778039   0.286308        Neutral    exon 1b    NaN   
4      0.548857          0.549808   0.000951   Intermediate    exon 1b    NaN   
...         ...               ...        ...            ...        ...    ...   
2263        NaN               NaN        NaN            NaN    exon 1c    NaN   
2264        NaN               NaN        NaN            NaN    exon 1c    NaN   
2265        NaN               NaN        NaN            NaN    exon 1c    NaN   
2266        NaN               NaN        NaN            NaN    exon 1c    NaN   
2267        NaN               NaN        NaN            NaN    exon 1c    NaN   

     Exon  cDNApos  CDSpos  Chrom          conseq oAA nAA        CCDS  \
0     1/3    951.0   111.0      3  NON_SYNONYMOUS   E   D  CCDS2597.1   
1     1/3    951.0   111.0      3  NON_SYNONYMOUS   E   D  CCDS2597.1   
2     1/3    952.0   112.0      3  NON_SYNONYMOUS   S   T  CCDS2597.1   
3     1/3    952.0   112.0      3  NON_SYNONYMOUS   S   A  CCDS2597.1   
4     1/3    953.0   113.0      3  NON_SYNONYMOUS   S   Y  CCDS2597.1   
...   ...      ...     ...    ...             ...  ..  ..         ...   
2263  1/3    949.0   109.0      3     STOP_GAINED   E   *  CCDS2597.1   
2264  1/3    940.0   100.0      3  NON_SYNONYMOUS   G   R  CCDS2597.1   
2265  1/3    944.0   104.0      3  NON_SYNONYMOUS   A   D  CCDS2597.1   
2266  1/3    929.0    89.0      3  NON_SYNONYMOUS   G   V  CCDS2597.1   
2267  1/3    944.0   104.0      3  NON_SYNONYMOUS   A   V  CCDS2597.1   

      protPos                                            variant cigar  \
0        37.0  GAGTACGGCCCTGAAGAAGACGGCGGGGAGGAGTCGGGCGCCGAGG...  214M   
1        37.0  GAGTACGGCCCTGAAGAAGACGGCGGGGAGGAGTCGGGCGCCGAGG...  214M   
2        38.0  GAGTACGGCCCTGAAGAAGACGGCGGGGAGGAGTCGGGCGCCGAGG...  214M   
3        38.0  GAGTACGGCCCTGAAGAAGACGGCGGGGAGGAGTCGGGCGCCGAGG...  214M   
4        38.0  GAGTACGGCCCTGAAGAAGACGGCGGGGAGGAGTCGGGCGCCGAGG...  214M   
...       ...                                                ...   ...   
2263     37.0  TACGGAGGTCGACTCGGGAGCGCGCACGCAGCTCCGCCCCGCGTCC...  258M   
2264     34.0  TACGGAGGTCGACTCGGGAGCGCGCACGCAGCTCCGCCCCGCGTCC...  258M   
2265     35.0  TACGGAGGTCGACTCGGGAGCGCGCACGCAGCTCCGCCCCGCGTCC...  258M   
2266     30.0  TACGGAGGTCGACTCGGGAGCGCGCACGCAGCTCCGCCCCGCGTCC...  258M   
2267     35.0  TACGGAGGTCGACTCGGGAGCGCGCACGCAGCTCCGCCCCGCGTCC...  258M   

                                            edit_string  tHDR_pre  tHDR_post  \
0                                  48-X-C,72-X-C,87-X-A       503        735   
1                                  48-X-T,72-X-C,87-X-A       581        923   
2                                  49-X-A,72-X-C,87-X-A       394        442   
3                                 

In [5]:
from Bio.PDB.Polypeptide import protein_letters_3to1
protein_letters_1to3 = {v: k.title() for k, v in protein_letters_3to1.items()}
protein_letters_1to3['*'] = "Ter"
def make_hgvs_pro(s):
    return f"p.{protein_letters_1to3[s[2]]}{s[3:-1]}{protein_letters_1to3[s[-1]]}"

In [6]:
data = data.dropna(subset=['pHGVS',])
data = data.assign(hgvs_pro=data.pHGVS.apply(make_hgvs_pro)).rename(columns=dict(function_score_final='score'))

In [7]:
data

POSITION REF ALT     cHGVS   pHGVS     score   q_value  rna_score  \
0     10141958   G   C  c.111G>C  p.E37D  0.227104  0.999801   0.146014   
1     10141958   G   T  c.111G>T  p.E37D  0.480106  0.999801   0.496812   
2     10141959   T   A  c.112T>A  p.S38T -0.065021  0.736202  -0.072035   
3     10141959   T   G  c.112T>G  p.S38A  0.235124  0.999801  -2.064347   
4     10141960   C   A  c.113C>A  p.S38Y -0.324690  0.124480   0.548857   
...        ...  ..  ..       ...     ...       ...       ...        ...   
2263  10141956   G   T  c.109G>T  p.E37*  1.197395  0.986207        NaN   
2264  10141947   G   C  c.100G>C  p.G34R  1.199658  0.986207        NaN   
2265  10141951   C   A  c.104C>A  p.A35D  1.452510  0.986207        NaN   
2266  10141936   G   T   c.89G>T  p.G30V  1.506749  0.986207        NaN   
2267  10141951   C   T  c.104C>T  p.A35V  1.525030  0.986207        NaN   

      rna_score_day_20  delta_rna function_class sge_region Intron Exon  \
0             0.049020  -0.096993        Neutral    exon 1b    NaN  1/3   
1             0.314450  -0.182362        Neutral    exon 1b    NaN  1/3   
2            -0.281796  -0.209761        Neutral    exon 1b    NaN  1/3   
3            -1.778039   0.286308        Neutral    exon 1b    NaN  1/3   
4             0.549808   0.000951   Intermediate    exon 1b    NaN  1/3   
...                ...        ...            ...        ...    ...  ...   
2263               NaN        NaN            NaN    exon 1c    NaN  1/3   
2264               NaN        NaN            NaN    exon 1c    NaN  1/3   
2265               NaN        NaN            NaN    exon 1c    NaN  1/3   
2266               NaN        NaN            NaN    exon 1c    NaN  1/3   
2267               NaN        NaN            NaN    exon 1c    NaN  1/3   

      cDNApos  CDSpos  Chrom          conseq oAA nAA        CCDS  protPos  \
0       951.0   111.0      3  NON_SYNONYMOUS   E   D  CCDS2597.1     37.0   
1       951.0   111.0      3  NON_SYNONYMOUS   E   D  CCDS2597.1     37.0   
2       952.0   112.0      3  NON_SYNONYMOUS   S   T  CCDS2597.1     38.0   
3       952.0   112.0      3  NON_SYNONYMOUS   S   A  CCDS2597.1     38.0   
4       953.0   113.0      3  NON_SYNONYMOUS   S   Y  CCDS2597.1     38.0   
...       ...     ...    ...             ...  ..  ..         ...      ...   
2263    949.0   109.0      3     STOP_GAINED   E   *  CCDS2597.1     37.0   
2264    940.0   100.0      3  NON_SYNONYMOUS   G   R  CCDS2597.1     34.0   
2265    944.0   104.0      3  NON_SYNONYMOUS   A   D  CCDS2597.1     35.0   
2266    929.0    89.0      3  NON_SYNONYMOUS   G   V  CCDS2597.1     30.0   
2267    944.0   104.0      3  NON_SYNONYMOUS   A   V  CCDS2597.1     35.0   

                                                variant cigar  \
0     GAGTACGGCCCTGAAGAAGACGGCGGGGAGGAGTCGGGCGCCGAGG...  214M   
1     GAGTACGGCCCTGAAGAAGACGGCGGGGAGGAGTCGGGCGCCGAGG...  214M   
2     GAGTACGGCCCTGAAGAAGACGGCGGGGAGGAGTCGGGCGCCGAGG...  214M   
3     GAGTACGGCCCTGAAGAAGACGGCGGGGAGGAGTCGGGCGCCGAGG...  214M   
4     GAGTACGGCCCTGAAGAAGACGGCGGGGAGGAGTCGGGCGCCGAGG...  214M   
...                                                 ...   ...   
2263  TACGGAGGTCGACTCGGGAGCGCGCACGCAGCTCCGCCCCGCGTCC...  258M   
2264  TACGGAGGTCGACTCGGGAGCGCGCACGCAGCTCCGCCCCGCGTCC...  258M   
2265  TACGGAGGTCGACTCGGGAGCGCGCACGCAGCTCCGCCCCGCGTCC...  258M   
2266  TACGGAGGTCGACTCGGGAGCGCGCACGCAGCTCCGCCCCGCGTCC...  258M   
2267  TACGGAGGTCGACTCGGGAGCGCGCACGCAGCTCCGCCCCGCGTCC...  258M   

                                            edit_string  tHDR_pre  tHDR_post  \
0                                  48-X-C,72-X-C,87-X-A       503        735   
1                                  48-X-T,72-X-C,87-X-A       581        923   
2                                  49-X-A,72-X-C,87-X-A       394        442   
3                                  49-X-G,72-X-C,87-X-A       483        498   
4                                  50-X-A,72-X-C,87-X-A       366        411   
...               

In [8]:
data.to_csv(dataset_path / "scoreset.csv", index=False)

In [27]:
metadata = dict(uniprot_acc='P40337')
with open(dataset_path / "metadata.json", "w") as f:
    json.dump(metadata, f)